In [1]:
# Author: Kaylani Bochie
# github.com/kaylani2
# kaylani AT gta DOT ufrj DOT br

### K: Model: Autoencoder
import sys
import time
import pandas as pd
import os
import math
sys.path.insert(1, '../')
import numpy as np
from numpy import mean, std
from unit import remove_columns_with_one_value, remove_nan_columns, load_dataset
from unit import display_general_information, display_feature_distribution
from collections import Counter
#from imblearn.over_sampling import RandomOverSampler, RandomUnderSampler
import sklearn
from sklearn import set_config
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, PredefinedSplit, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif
from sklearn.utils import class_weight
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import keras.utils
from keras import metrics
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM
from keras.optimizers import RMSprop, Adam
from keras.constraints import maxnorm

In [2]:
###############################################################################
## Define constants
###############################################################################
pd.set_option ('display.max_rows', None)
pd.set_option ('display.max_columns', 5)
BOT_IOT_DIRECTORY = '../../../../../datasets/bot-iot/'
BOT_IOT_FEATURE_NAMES = 'UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv'
BOT_IOT_FILE_5_PERCENT_SCHEMA = 'UNSW_2018_IoT_Botnet_Full5pc_{}.csv' # 1 - 4
FIVE_PERCENT_FILES = 4
BOT_IOT_FILE_FULL_SCHEMA = 'UNSW_2018_IoT_Botnet_Dataset_{}.csv' # 1 - 74
FULL_FILES = 74
FILE_NAME = BOT_IOT_DIRECTORY + BOT_IOT_FILE_5_PERCENT_SCHEMA
FEATURES = BOT_IOT_DIRECTORY + BOT_IOT_FEATURE_NAMES
NAN_VALUES = ['?', '.']
TARGET = 'attack'
INDEX_COLUMN = 'pkSeqID'
LABELS = ['attack', 'category', 'subcategory']
STATE = 0
try:
  STATE = int (sys.argv [1])
except:
  pass
#for STATE in [1, 2, 3, 4, 5]:
np.random.seed (STATE)
print ('STATE:', STATE)

STATE: 0


In [3]:
###############################################################################
## Load dataset
###############################################################################
df = load_dataset (FILE_NAME, FIVE_PERCENT_FILES, INDEX_COLUMN, NAN_VALUES)

Reading ../../../../../datasets/bot-iot/UNSW_2018_IoT_Botnet_Full5pc_1.csv
Reading ../../../../../datasets/bot-iot/UNSW_2018_IoT_Botnet_Full5pc_2.csv
Reading ../../../../../datasets/bot-iot/UNSW_2018_IoT_Botnet_Full5pc_3.csv
Reading ../../../../../datasets/bot-iot/UNSW_2018_IoT_Botnet_Full5pc_4.csv


In [4]:
###############################################################################
## Clean dataset
###############################################################################
###############################################################################
### Remove columns with only one value
df, log = remove_columns_with_one_value (df, verbose = False)
print (log)


###############################################################################
### Remove redundant columns, useless columns and unused targets
### K: _number columns are numerical representations of other existing columns.
### K: category and subcategory are other labels.
### K: saddr and daddr may specialize the model to a single network
redundant_columns = ['state_number', 'proto_number', 'flgs_number']
other_targets = ['category', 'subcategory']
misc_columns = ['saddr', 'daddr']
print ('Removing redundant columns:', redundant_columns)
print ('Removing useless targets:', other_targets)
print ('Removing misc columns:', misc_columns)
columns_to_remove = redundant_columns + other_targets + misc_columns
df.drop (axis = 'columns', columns = columns_to_remove, inplace = True)

###############################################################################
### Remove NaN columns (with a lot of NaN values)
df, log = remove_nan_columns (df, 1/2, verbose = False)
print (log)

###############################################################################
### Encode categorical features
print ('Encoding categorical features (ordinal encoding).')
my_encoder = OrdinalEncoder ()
df ['flgs'] = my_encoder.fit_transform (df ['flgs'].values.reshape (-1, 1))
df ['proto'] = my_encoder.fit_transform (df ['proto'].values.reshape (-1, 1))
df ['sport'] = my_encoder.fit_transform (df ['sport'].astype (str).values.reshape (-1, 1))
df ['dport'] = my_encoder.fit_transform (df ['dport'].astype (str).values.reshape (-1, 1))
df ['state'] = my_encoder.fit_transform (df ['state'].values.reshape (-1, 1))
print ('Objects:', list (df.select_dtypes ( ['object']).columns))

While removing single value columns: No columns dropped.
Removing redundant columns: ['state_number', 'proto_number', 'flgs_number']
Removing useless targets: ['category', 'subcategory']
Removing misc columns: ['saddr', 'daddr']
While removing nan value columns: No columns dropped.
Encoding categorical features (ordinal encoding).
Objects: []


In [5]:
###############################################################################
## Quick sanity check
###############################################################################
display_general_information (df)

Dataframe shape (lines, columns): (3668522, 38) 

First 5 entries:
                 stime  flgs  ...  Pkts_P_State_P_Protocol_P_SrcIP  attack
pkSeqID                      ...                                         
1        1.528089e+09   0.0  ...                              602       1
2        1.528089e+09   0.0  ...                                6       1
3        1.528089e+09   0.0  ...                              602       1
4        1.528089e+09   0.0  ...                              602       1
5        1.528089e+09   0.0  ...                              602       1

[5 rows x 38 columns] 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3668522 entries, 1 to 3668522
Data columns (total 38 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   stime                             float64
 1   flgs                              float64
 2   proto                             float64
 3   sport                             fl

In [6]:
###############################################################################
## Split dataset
###############################################################################
## K: Dataset is too big? Drop.
drop_indices = np.random.choice (df.index, int (df.shape [0] * 0.5),
                                 replace = False)

In [7]:
### Isolate attack and normal samples
df = df.drop (drop_indices)
mask = df [TARGET] == 0
# 0 == normal
df_normal = df [mask]
# 1 == attack
df_attack = df [~mask]

print ('Attack set:')
print (df_attack [TARGET].value_counts ())
print ('Normal set:')
print (df_normal [TARGET].value_counts ())

Attack set:
1    1834017
Name: attack, dtype: int64
Normal set:
0    244
Name: attack, dtype: int64


In [8]:
### Sample and drop random attacks
df_random_attacks = df_attack.sample (n = df_normal.shape [0], random_state = STATE)
df_attack = df_attack.drop (df_random_attacks.index)

### Assemble train set (only attacks)
X_train = df_attack.loc [:, df.columns != TARGET]
y_train = df_attack [TARGET]
print ('Train set:')
print (df_attack [TARGET].value_counts ())

### Assemble test set (50/50 attacks and non-attacks)
df_test = pd.DataFrame ()
df_test = pd.concat ( [df_test, df_normal])
df_test = pd.concat ( [df_test, df_random_attacks])
print ('Test set:')
print (df_test [TARGET].value_counts ())
X_test = df_test.loc [:, df.columns != TARGET]
y_test = df_test [TARGET]
### K: y_test is required to plot the roc curve in the end

# df_train = df_attack
VALIDATION_SIZE = 1/4
print ('\nSplitting dataset (validation/train):', VALIDATION_SIZE)
X_train, X_val, y_train, y_val = train_test_split (
                                             X_train,
                                             y_train,
                                             test_size = VALIDATION_SIZE,
                                             random_state = STATE,)

Train set:
1    1833773
Name: attack, dtype: int64
Test set:
1    244
0    244
Name: attack, dtype: int64

Splitting dataset (validation/train): 0.25


In [9]:
print ('X_train shape:', X_train.shape)
print ('y_train shape:', y_train.shape)
print ('X_val shape:', X_val.shape)
print ('y_val shape:', y_val.shape)
print ('X_test shape:', X_test.shape)
print ('y_test shape:', y_test.shape)

X_train shape: (1375329, 37)
y_train shape: (1375329,)
X_val shape: (458444, 37)
y_val shape: (458444,)
X_test shape: (488, 37)
y_test shape: (488,)


In [10]:
# ###############################################################################
# ## Convert dataframe to a numpy array
# ###############################################################################
# print ('\nConverting dataframe to numpy array.')
# X_train = X_train.values
# y_train = y_train.values
# X_val = X_val.values
# y_val = y_val.values
# X_test = X_test.values
# y_test = y_test.values
# print ('X_train shape:', X_train.shape)
# print ('y_train shape:', y_train.shape)
# print ('X_val shape:', X_val.shape)
# print ('y_val shape:', y_val.shape)
# print ('X_test shape:', X_test.shape)
# print ('y_test shape:', y_test.shape)

In [11]:
###############################################################################
## Apply normalization
###############################################################################
### K: NOTE: Only use derived information from the train set to avoid leakage.
print ('\nApplying normalization.')
startTime = time.time ()
scaler = StandardScaler ()
scaler.fit (X_train)
X_train = scaler.transform (X_train)
X_val = scaler.transform (X_val)
X_test = scaler.transform (X_test)
print (str (time.time () - startTime), 'to normalize data.')


Applying normalization.
1.311103343963623 to normalize data.


In [12]:
print ('X_train shape:', X_train.shape)
print ('y_train shape:', y_train.shape)
print ('X_val shape:', X_val.shape)
print ('y_val shape:', y_val.shape)
print ('X_test shape:', X_test.shape)
print ('y_test shape:', y_test.shape)
print (type (X_train))

X_train shape: (1375329, 37)
y_train shape: (1375329,)
X_val shape: (458444, 37)
y_val shape: (458444,)
X_test shape: (488, 37)
y_test shape: (488,)
<class 'numpy.ndarray'>


In [20]:
###############################################################################
## Perform feature selection
###############################################################################
### K: TODO: Adjust NUMBER_OF_FEATURES
NUMBER_OF_FEATURES = 15 #X_train.shape [-1]
print ('\nSelecting top', NUMBER_OF_FEATURES, 'features.')
fs = PCA (n_components = NUMBER_OF_FEATURES)
fs.fit (X_train)
print (fs.explained_variance_ratio_)
print (fs.singular_values_)
X_train = fs.transform (X_train)
X_val = fs.transform (X_val)
X_test = fs.transform (X_test)
print ('X_train shape:', X_train.shape)
print ('y_train shape:', y_train.shape)
print ('X_val shape:', X_val.shape)
print ('y_val shape:', y_val.shape)
print ('X_test shape:', X_test.shape)
print ('y_test shape:', y_test.shape)


Selecting top 15 features.
[0.29279687 0.11136005 0.09905962 0.06917073 0.05745066 0.04058962
 0.03858393 0.03404088 0.03279992 0.02985979 0.02816583 0.02643571
 0.0254068  0.02318551 0.01827428]
[3859.71897053 2380.33006749 2245.02297231 1876.0035388  1709.70027251
 1437.07671729 1401.12113079 1316.05147808 1291.8402985  1232.58215299
 1197.1091184  1159.75950706 1136.96583007 1086.12746759  964.25655449]
X_train shape: (1375329, 15)
y_train shape: (1375329,)
X_val shape: (458444, 15)
y_val shape: (458444,)
X_test shape: (488, 15)
y_test shape: (488,)


In [63]:
###############################################################################
## Create learning model (Autoencoder) and tune hyperparameters
###############################################################################
###############################################################################
# Hyperparameter tuning
test_fold = np.repeat ([-1, 0], [X_train.shape [0], X_val.shape [0]])
myPreSplit = PredefinedSplit (test_fold)
def create_model (learn_rate = 0.01, dropout_rate = 0.0, weight_constraint = 0,
                  metrics = ['mse'], input_layer_neurons,
                  neurons_on_first_layer = 32,
                  second_layer_boolean = False,
                  neurons_on_chokehold_layer = 8):
  model = Sequential ()
  model.add (Dense (input_layer_neurons, activation = 'relu',
                   input_shape = (input_layer_neurons, )))
  if (second_layer_boolean):
    model.add (Dense (neurons_on_first_layer/2, activation = 'relu'))
  model.add (Dense (neurons_on_second_layer, activation = 'relu'))
  model.add (Dense (neurons_on_chokehold_layer,  activation = 'relu'))
  if (second_layer_boolean):
    model.add (Dense (neurons_on_first_layer/2, activation = 'relu'))
  model.add (Dense (neurons_on_first_layer, activation = 'relu'))
  model.add (Dense (input_layer_neurons, activation = None))
  model.compile (loss = 'mean_squared_error',
                 optimizer = 'adam',
                 metrics = metrics)
  return model


model = KerasRegressor (build_fn = create_model, verbose = 2)
input_layer_neurons = X_train.shape [1]
batch_size = [5000, 10000]
epochs = [200]
learn_rate = [0.0001, 0.001]
dropout_rate = [0.0]
weight_constraint = [0]
neurons_on_first_layer = [32, 64]
neurons_on_chokehold_layer = [4, 8]
second_layer_boolean = [False, True] # Is there another layer?
param_grid = dict (batch_size = batch_size, epochs = epochs,
                   dropout_rate = dropout_rate, learn_rate = learn_rate,
                   weight_constraint = weight_constraint,
                   input_layer_neurons = input_layer_neurons,
                   neurons_on_first_layer = neurons_on_first_layer,
                   second_layer_boolean = second_layer_boolean,
                   neurons_on_chokehold_layer = neurons_on_chokehold_layer)
grid = GridSearchCV (estimator = model, param_grid = param_grid,
                    scoring = 'neg_mean_squared_error', cv = myPreSplit,
                    verbose = 2, n_jobs = 1)
startTime = time.time ()
grid_result = grid.fit (np.vstack ( (X_train, X_val)),#, axis = 1),
                       np.vstack ( (X_train, X_val)))#, axis = 1))
print (str (time.time () - startTime), 's to search grid.')
print (grid_result.best_params_)

print ("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_ ['mean_test_score']
stds = grid_result.cv_results_ ['std_test_score']
params = grid_result.cv_results_ ['params']
for mean, stdev, param in zip (means, stds, params):
  print ("%f (%f) with: %r" % (mean, stdev, param))

## 9 features:
## Best: -0.148847 using {'batch_size': 5000, 'dropout_rate': 0.0, 'epochs': 10, 'learn_rate': 0.001, 'weight_constraint': 0}

## All features (SEM PCA): (97s to search grid)
## Best: -0.159944 using {'batch_size': 10000, 'dropout_rate': 0.0, 'epochs': 10, 'learn_rate': 0.1, 'weight_constraint': 0}

## All features (COM PCA):  (100s to search grid)
## Best: -0.100227 using {'batch_size': 5000, 'dropout_rate': 0.0, 'epochs': 10, 'learn_rate': 0.001, 'weight_constraint': 0}

## 32 features: (88s to search grid)
## Best: -0.179934 using {'batch_size': 5000, 'dropout_rate': 0.0, 'epochs': 10, 'learn_rate': 0.001, 'weight_constraint': 0}

## 15 features (67s to search grid)
## Best: -0.235642 using {'batch_size': 10000, 'dropout_rate': 0.0, 'epochs': 10, 'learn_rate': 0.1, 'weight_constraint': 0}

## 15 features (60min to search grid)
## Best: -0.019674 using {'batch_size': 10000, 'dropout_rate': 0.0, 'epochs': 200, 'learn_rate': 0.0001, 'neurons_on_chokehold_layer': 8, 'neurons_on_first_layer': 64, 'weight_constraint': 0}

Fitting 1 folds for each of 16 candidates, totalling 16 fits
[CV] batch_size=5000, dropout_rate=0.0, epochs=200, learn_rate=0.0001, neurons_on_chokehold_layer=4, neurons_on_first_layer=32, weight_constraint=0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/200
276/276 - 1s - loss: 1.4657 - mse: 1.4657
Epoch 2/200
276/276 - 1s - loss: 0.9250 - mse: 0.9250
Epoch 3/200
276/276 - 1s - loss: 0.7840 - mse: 0.7840
Epoch 4/200
276/276 - 1s - loss: 0.7003 - mse: 0.7003
Epoch 5/200
276/276 - 1s - loss: 0.6602 - mse: 0.6602
Epoch 6/200
276/276 - 1s - loss: 0.5754 - mse: 0.5754
Epoch 7/200
276/276 - 1s - loss: 0.5150 - mse: 0.5150
Epoch 8/200
276/276 - 1s - loss: 0.4680 - mse: 0.4680
Epoch 9/200
276/276 - 1s - loss: 0.4426 - mse: 0.4426
Epoch 10/200
276/276 - 1s - loss: 0.4311 - mse: 0.4311
Epoch 11/200
276/276 - 1s - loss: 0.4078 - mse: 0.4078
Epoch 12/200
276/276 - 1s - loss: 0.3663 - mse: 0.3663
Epoch 13/200
276/276 - 1s - loss: 0.3455 - mse: 0.3455
Epoch 14/200
276/276 - 1s - loss: 0.3388 - mse: 0.3388
Epoch 15/200
276/276 - 1s - loss: 0.3326 - mse: 0.3326
Epoch 16/200
276/276 - 1s - loss: 0.3315 - mse: 0.3315
Epoch 17/200
276/276 - 1s - loss: 0.3374 - mse: 0.3374
Epoch 18/200
276/276 - 1s - loss: 0.3477 - mse: 0.3477
Epoch 19/200
276/27

276/276 - 1s - loss: 0.0958 - mse: 0.0958
Epoch 150/200
276/276 - 1s - loss: 0.0921 - mse: 0.0921
Epoch 151/200
276/276 - 1s - loss: 0.0936 - mse: 0.0936
Epoch 152/200
276/276 - 1s - loss: 0.0943 - mse: 0.0943
Epoch 153/200
276/276 - 1s - loss: 0.0961 - mse: 0.0961
Epoch 154/200
276/276 - 1s - loss: 0.1144 - mse: 0.1144
Epoch 155/200
276/276 - 1s - loss: 0.1100 - mse: 0.1100
Epoch 156/200
276/276 - 1s - loss: 0.1086 - mse: 0.1086
Epoch 157/200
276/276 - 1s - loss: 0.1074 - mse: 0.1074
Epoch 158/200
276/276 - 1s - loss: 0.1012 - mse: 0.1012
Epoch 159/200
276/276 - 1s - loss: 0.1077 - mse: 0.1077
Epoch 160/200
276/276 - 1s - loss: 0.0933 - mse: 0.0933
Epoch 161/200
276/276 - 1s - loss: 0.0926 - mse: 0.0926
Epoch 162/200
276/276 - 1s - loss: 0.1014 - mse: 0.1014
Epoch 163/200
276/276 - 1s - loss: 0.0967 - mse: 0.0967
Epoch 164/200
276/276 - 1s - loss: 0.1019 - mse: 0.1019
Epoch 165/200
276/276 - 1s - loss: 0.1023 - mse: 0.1023
Epoch 166/200
276/276 - 1s - loss: 0.0968 - mse: 0.0968
Epoch 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


...

Epoch 140/200
138/138 - 1s - loss: 0.0311 - mse: 0.0311
Epoch 141/200
138/138 - 1s - loss: 0.0195 - mse: 0.0195
Epoch 142/200
138/138 - 1s - loss: 0.0203 - mse: 0.0203
Epoch 143/200
138/138 - 1s - loss: 0.0166 - mse: 0.0166
Epoch 144/200
138/138 - 1s - loss: 0.0150 - mse: 0.0150
Epoch 145/200
138/138 - 1s - loss: 0.0153 - mse: 0.0153
Epoch 146/200
138/138 - 1s - loss: 0.0148 - mse: 0.0148
Epoch 147/200
138/138 - 1s - loss: 0.0150 - mse: 0.0150
Epoch 148/200
138/138 - 1s - loss: 0.0155 - mse: 0.0155
Epoch 149/200
138/138 - 1s - loss: 0.0229 - mse: 0.0229
Epoch 150/200
138/138 - 1s - loss: 0.0220 - mse: 0.0220
Epoch 151/200
138/138 - 1s - loss: 0.0168 - mse: 0.0168
Epoch 152/200
138/138 - 1s - loss: 0.0151 - mse: 0.0151
Epoch 153/200
138/138 - 1s - loss: 0.0139 - mse: 0.0139
Epoch 154/200
138/138 - 1s - loss: 0.0142 - mse: 0.0142
Epoch 155/200
138/138 - 1s - loss: 0.0151 - mse: 0.0151
Epoch 156/200
138/138 - 1s - loss: 0.0214 - mse: 0.0214
Epoch 157/200
138/138 - 1s - loss: 0.0316 - mse:

138/138 - 1s - loss: 0.0366 - mse: 0.0366
Epoch 83/200
138/138 - 1s - loss: 0.0268 - mse: 0.0268
Epoch 84/200
138/138 - 1s - loss: 0.0247 - mse: 0.0247
Epoch 85/200
138/138 - 1s - loss: 0.0242 - mse: 0.0242
Epoch 86/200
138/138 - 1s - loss: 0.0278 - mse: 0.0278
Epoch 87/200
138/138 - 1s - loss: 0.0259 - mse: 0.0259
Epoch 88/200
138/138 - 1s - loss: 0.0341 - mse: 0.0341
Epoch 89/200
138/138 - 1s - loss: 0.0354 - mse: 0.0354
Epoch 90/200
138/138 - 1s - loss: 0.0336 - mse: 0.0336
Epoch 91/200
138/138 - 1s - loss: 0.0214 - mse: 0.0214
Epoch 92/200
138/138 - 1s - loss: 0.0278 - mse: 0.0278
Epoch 93/200
138/138 - 1s - loss: 0.0241 - mse: 0.0241
Epoch 94/200
138/138 - 1s - loss: 0.0261 - mse: 0.0261
Epoch 95/200
138/138 - 1s - loss: 0.0290 - mse: 0.0290
Epoch 96/200
138/138 - 1s - loss: 0.0276 - mse: 0.0276
Epoch 97/200
138/138 - 1s - loss: 0.0258 - mse: 0.0258
Epoch 98/200
138/138 - 1s - loss: 0.0345 - mse: 0.0345
Epoch 99/200
138/138 - 1s - loss: 0.0237 - mse: 0.0237
Epoch 100/200
138/138 -

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed: 58.7min finished


184/184 - 1s - loss: 1.3183 - mse: 1.3183
Epoch 2/200
184/184 - 1s - loss: 0.6339 - mse: 0.6339
Epoch 3/200
184/184 - 1s - loss: 0.4502 - mse: 0.4502
Epoch 4/200
184/184 - 1s - loss: 0.3294 - mse: 0.3294
Epoch 5/200
184/184 - 1s - loss: 0.2426 - mse: 0.2426
Epoch 6/200
184/184 - 1s - loss: 0.2086 - mse: 0.2086
Epoch 7/200
184/184 - 1s - loss: 0.1593 - mse: 0.1593
Epoch 8/200
184/184 - 2s - loss: 0.1460 - mse: 0.1460
Epoch 9/200
184/184 - 2s - loss: 0.1209 - mse: 0.1209
Epoch 10/200
184/184 - 2s - loss: 0.1115 - mse: 0.1115
Epoch 11/200
184/184 - 2s - loss: 0.0987 - mse: 0.0987
Epoch 12/200
184/184 - 2s - loss: 0.0854 - mse: 0.0854
Epoch 13/200
184/184 - 2s - loss: 0.0906 - mse: 0.0906
Epoch 14/200
184/184 - 2s - loss: 0.0913 - mse: 0.0913
Epoch 15/200
184/184 - 2s - loss: 0.0693 - mse: 0.0693
Epoch 16/200
184/184 - 2s - loss: 0.0663 - mse: 0.0663
Epoch 17/200
184/184 - 2s - loss: 0.0823 - mse: 0.0823
Epoch 18/200
184/184 - 2s - loss: 0.0666 - mse: 0.0666
Epoch 19/200
184/184 - 2s - los

Epoch 150/200
184/184 - 2s - loss: 0.0482 - mse: 0.0482
Epoch 151/200
184/184 - 2s - loss: 0.0205 - mse: 0.0205
Epoch 152/200
184/184 - 2s - loss: 0.0102 - mse: 0.0102
Epoch 153/200
184/184 - 2s - loss: 0.0094 - mse: 0.0094
Epoch 154/200
184/184 - 2s - loss: 0.0099 - mse: 0.0099
Epoch 155/200
184/184 - 2s - loss: 0.0106 - mse: 0.0106
Epoch 156/200
184/184 - 2s - loss: 0.0116 - mse: 0.0116
Epoch 157/200
184/184 - 2s - loss: 0.0134 - mse: 0.0134
Epoch 158/200
184/184 - 2s - loss: 0.0128 - mse: 0.0128
Epoch 159/200
184/184 - 2s - loss: 0.0113 - mse: 0.0113
Epoch 160/200
184/184 - 2s - loss: 0.0093 - mse: 0.0093
Epoch 161/200
184/184 - 2s - loss: 0.0090 - mse: 0.0090
Epoch 162/200
184/184 - 2s - loss: 0.0165 - mse: 0.0165
Epoch 163/200
184/184 - 2s - loss: 0.0289 - mse: 0.0289
Epoch 164/200
184/184 - 2s - loss: 0.0168 - mse: 0.0168
Epoch 165/200
184/184 - 2s - loss: 0.0113 - mse: 0.0113
Epoch 166/200
184/184 - 2s - loss: 0.0129 - mse: 0.0129
Epoch 167/200
184/184 - 2s - loss: 0.0142 - mse:

In [38]:
###############################################################################
## Finished model
# Best: -0.015576 using {'batch_size': 5000, 'dropout_rate': 0.0, 'epochs': 10,
# learn_rate': 0.1, 'weight_constraint': 0}
METRICS = [keras.metrics.MeanSquaredError (name = 'MSE'),
           keras.metrics.RootMeanSquaredError (name = 'RMSE'),  
           keras.metrics.MeanAbsoluteError (name = 'MAE'),]
### K: learning rate foi alterado manualmente ao olhar os valores do erro na
### validacao ao longo das epochs...
NUMBER_OF_EPOCHS = 700
BATCH_SIZE = 10000
LEARNING_RATE = 0.0001

print ('\nCreating learning model.')
clf = Sequential ()
clf.add (Dense (X_train.shape [1], activation = 'relu',
                      input_shape = (X_train.shape [1], )))
clf.add (Dense (32, activation = 'relu'))
clf.add (Dense (8,  activation = 'relu'))
clf.add (Dense (32, activation = 'relu'))
clf.add (Dense (X_train.shape [1], activation = None))


###############################################################################
## Compile the network
###############################################################################
print ('\nCompiling the network.')
clf.compile (loss = 'mean_squared_error',
                   optimizer = Adam (lr = LEARNING_RATE),
                   metrics = METRICS)
print ('Model summary:')
clf.summary ()


Creating learning model.

Compiling the network.
Model summary:
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_180 (Dense)            (None, 15)                240       
_________________________________________________________________
dense_181 (Dense)            (None, 32)                512       
_________________________________________________________________
dense_182 (Dense)            (None, 8)                 264       
_________________________________________________________________
dense_183 (Dense)            (None, 32)                288       
_________________________________________________________________
dense_184 (Dense)            (None, 15)                495       
Total params: 1,799
Trainable params: 1,799
Non-trainable params: 0
_________________________________________________________________


In [39]:
###############################################################################
## Fit the network
###############################################################################
print ('\nFitting the network.')
startTime = time.time ()
history = clf.fit (X_train, X_train,
                         batch_size = BATCH_SIZE,
                         epochs = NUMBER_OF_EPOCHS,
                         verbose = 2, #1 = progress bar, not useful for logging
                         workers = 0,
                         use_multiprocessing = True,
                         #class_weight = 'auto',
                         validation_data = (X_val, X_val))
print (str (time.time () - startTime), 's to train model.')


Fitting the network.
Epoch 1/700
138/138 - 1s - loss: 2.2709 - MSE: 2.2709 - RMSE: 1.5069 - MAE: 0.7962 - val_loss: 2.6674 - val_MSE: 2.6674 - val_RMSE: 1.6332 - val_MAE: 0.7907
Epoch 2/700
138/138 - 1s - loss: 2.1862 - MSE: 2.1862 - RMSE: 1.4786 - MAE: 0.7843 - val_loss: 2.5685 - val_MSE: 2.5685 - val_RMSE: 1.6027 - val_MAE: 0.7775
Epoch 3/700
138/138 - 1s - loss: 2.0567 - MSE: 2.0567 - RMSE: 1.4341 - MAE: 0.7648 - val_loss: 2.3964 - val_MSE: 2.3964 - val_RMSE: 1.5480 - val_MAE: 0.7507
Epoch 4/700
138/138 - 1s - loss: 1.8582 - MSE: 1.8582 - RMSE: 1.3632 - MAE: 0.7319 - val_loss: 2.1681 - val_MSE: 2.1681 - val_RMSE: 1.4724 - val_MAE: 0.7098
Epoch 5/700
138/138 - 1s - loss: 1.6246 - MSE: 1.6246 - RMSE: 1.2746 - MAE: 0.6809 - val_loss: 1.9257 - val_MSE: 1.9257 - val_RMSE: 1.3877 - val_MAE: 0.6501
Epoch 6/700
138/138 - 1s - loss: 1.4148 - MSE: 1.4148 - RMSE: 1.1894 - MAE: 0.6217 - val_loss: 1.7394 - val_MSE: 1.7394 - val_RMSE: 1.3189 - val_MAE: 0.5968
Epoch 7/700
138/138 - 1s - loss: 1.2

Epoch 53/700
138/138 - 1s - loss: 0.2436 - MSE: 0.2436 - RMSE: 0.4936 - MAE: 0.2057 - val_loss: 0.4172 - val_MSE: 0.4172 - val_RMSE: 0.6459 - val_MAE: 0.2039
Epoch 54/700
138/138 - 1s - loss: 0.2382 - MSE: 0.2382 - RMSE: 0.4880 - MAE: 0.2027 - val_loss: 0.4087 - val_MSE: 0.4087 - val_RMSE: 0.6393 - val_MAE: 0.2016
Epoch 55/700
138/138 - 1s - loss: 0.2335 - MSE: 0.2335 - RMSE: 0.4832 - MAE: 0.2003 - val_loss: 0.4004 - val_MSE: 0.4004 - val_RMSE: 0.6327 - val_MAE: 0.1991
Epoch 56/700
138/138 - 1s - loss: 0.2283 - MSE: 0.2283 - RMSE: 0.4778 - MAE: 0.1977 - val_loss: 0.3923 - val_MSE: 0.3923 - val_RMSE: 0.6263 - val_MAE: 0.1966
Epoch 57/700
138/138 - 1s - loss: 0.2238 - MSE: 0.2238 - RMSE: 0.4730 - MAE: 0.1955 - val_loss: 0.3848 - val_MSE: 0.3848 - val_RMSE: 0.6203 - val_MAE: 0.1942
Epoch 58/700
138/138 - 1s - loss: 0.2194 - MSE: 0.2194 - RMSE: 0.4684 - MAE: 0.1931 - val_loss: 0.3776 - val_MSE: 0.3776 - val_RMSE: 0.6145 - val_MAE: 0.1920
Epoch 59/700
138/138 - 1s - loss: 0.2152 - MSE: 0.21

Epoch 105/700
138/138 - 1s - loss: 0.0979 - MSE: 0.0979 - RMSE: 0.3129 - MAE: 0.1389 - val_loss: 0.1649 - val_MSE: 0.1649 - val_RMSE: 0.4061 - val_MAE: 0.1389
Epoch 106/700
138/138 - 1s - loss: 0.0965 - MSE: 0.0965 - RMSE: 0.3106 - MAE: 0.1386 - val_loss: 0.1628 - val_MSE: 0.1628 - val_RMSE: 0.4035 - val_MAE: 0.1380
Epoch 107/700
138/138 - 1s - loss: 0.0946 - MSE: 0.0946 - RMSE: 0.3076 - MAE: 0.1378 - val_loss: 0.1589 - val_MSE: 0.1589 - val_RMSE: 0.3987 - val_MAE: 0.1373
Epoch 108/700
138/138 - 1s - loss: 0.0929 - MSE: 0.0929 - RMSE: 0.3047 - MAE: 0.1370 - val_loss: 0.1553 - val_MSE: 0.1553 - val_RMSE: 0.3940 - val_MAE: 0.1366
Epoch 109/700
138/138 - 1s - loss: 0.0912 - MSE: 0.0912 - RMSE: 0.3020 - MAE: 0.1360 - val_loss: 0.1520 - val_MSE: 0.1520 - val_RMSE: 0.3899 - val_MAE: 0.1366
Epoch 110/700
138/138 - 1s - loss: 0.0896 - MSE: 0.0896 - RMSE: 0.2993 - MAE: 0.1359 - val_loss: 0.1488 - val_MSE: 0.1488 - val_RMSE: 0.3858 - val_MAE: 0.1354
Epoch 111/700
138/138 - 1s - loss: 0.0879 - MS

Epoch 157/700
138/138 - 1s - loss: 0.0340 - MSE: 0.0340 - RMSE: 0.1845 - MAE: 0.1117 - val_loss: 0.0368 - val_MSE: 0.0368 - val_RMSE: 0.1917 - val_MAE: 0.1113
Epoch 158/700
138/138 - 1s - loss: 0.0336 - MSE: 0.0336 - RMSE: 0.1833 - MAE: 0.1113 - val_loss: 0.0358 - val_MSE: 0.0358 - val_RMSE: 0.1893 - val_MAE: 0.1111
Epoch 159/700
138/138 - 1s - loss: 0.0332 - MSE: 0.0332 - RMSE: 0.1822 - MAE: 0.1109 - val_loss: 0.0353 - val_MSE: 0.0353 - val_RMSE: 0.1878 - val_MAE: 0.1110
Epoch 160/700
138/138 - 1s - loss: 0.0328 - MSE: 0.0328 - RMSE: 0.1812 - MAE: 0.1106 - val_loss: 0.0345 - val_MSE: 0.0345 - val_RMSE: 0.1858 - val_MAE: 0.1101
Epoch 161/700
138/138 - 1s - loss: 0.0324 - MSE: 0.0324 - RMSE: 0.1801 - MAE: 0.1101 - val_loss: 0.0339 - val_MSE: 0.0339 - val_RMSE: 0.1842 - val_MAE: 0.1098
Epoch 162/700
138/138 - 1s - loss: 0.0321 - MSE: 0.0321 - RMSE: 0.1791 - MAE: 0.1097 - val_loss: 0.0335 - val_MSE: 0.0335 - val_RMSE: 0.1830 - val_MAE: 0.1094
Epoch 163/700
138/138 - 1s - loss: 0.0318 - MS

Epoch 209/700
138/138 - 1s - loss: 0.0243 - MSE: 0.0243 - RMSE: 0.1560 - MAE: 0.0955 - val_loss: 0.0248 - val_MSE: 0.0248 - val_RMSE: 0.1574 - val_MAE: 0.0951
Epoch 210/700
138/138 - 1s - loss: 0.0243 - MSE: 0.0243 - RMSE: 0.1557 - MAE: 0.0952 - val_loss: 0.0248 - val_MSE: 0.0248 - val_RMSE: 0.1574 - val_MAE: 0.0950
Epoch 211/700
138/138 - 1s - loss: 0.0242 - MSE: 0.0242 - RMSE: 0.1555 - MAE: 0.0950 - val_loss: 0.0248 - val_MSE: 0.0248 - val_RMSE: 0.1574 - val_MAE: 0.0950
Epoch 212/700
138/138 - 1s - loss: 0.0241 - MSE: 0.0241 - RMSE: 0.1552 - MAE: 0.0948 - val_loss: 0.0246 - val_MSE: 0.0246 - val_RMSE: 0.1569 - val_MAE: 0.0947
Epoch 213/700
138/138 - 1s - loss: 0.0240 - MSE: 0.0240 - RMSE: 0.1549 - MAE: 0.0946 - val_loss: 0.0245 - val_MSE: 0.0245 - val_RMSE: 0.1564 - val_MAE: 0.0946
Epoch 214/700
138/138 - 1s - loss: 0.0238 - MSE: 0.0238 - RMSE: 0.1543 - MAE: 0.0944 - val_loss: 0.0242 - val_MSE: 0.0242 - val_RMSE: 0.1555 - val_MAE: 0.0942
Epoch 215/700
138/138 - 1s - loss: 0.0238 - MS

...

Epoch 469/700
138/138 - 1s - loss: 0.0107 - MSE: 0.0107 - RMSE: 0.1033 - MAE: 0.0546 - val_loss: 0.0110 - val_MSE: 0.0110 - val_RMSE: 0.1050 - val_MAE: 0.0542
Epoch 470/700
138/138 - 1s - loss: 0.0107 - MSE: 0.0107 - RMSE: 0.1035 - MAE: 0.0545 - val_loss: 0.0111 - val_MSE: 0.0111 - val_RMSE: 0.1054 - val_MAE: 0.0542
Epoch 471/700
138/138 - 1s - loss: 0.0108 - MSE: 0.0108 - RMSE: 0.1037 - MAE: 0.0546 - val_loss: 0.0113 - val_MSE: 0.0113 - val_RMSE: 0.1064 - val_MAE: 0.0542
Epoch 472/700
138/138 - 1s - loss: 0.0108 - MSE: 0.0108 - RMSE: 0.1041 - MAE: 0.0546 - val_loss: 0.0114 - val_MSE: 0.0114 - val_RMSE: 0.1066 - val_MAE: 0.0543
Epoch 473/700
138/138 - 1s - loss: 0.0107 - MSE: 0.0107 - RMSE: 0.1036 - MAE: 0.0545 - val_loss: 0.0113 - val_MSE: 0.0113 - val_RMSE: 0.1062 - val_MAE: 0.0542
Epoch 474/700
138/138 - 1s - loss: 0.0106 - MSE: 0.0106 - RMSE: 0.1031 - MAE: 0.0541 - val_loss: 0.0111 - val_MSE: 0.0111 - val_RMSE: 0.1056 - val_MAE: 0.0588
Epoch 475/700
138/138 - 1s - loss: 0.0108 - MS

Epoch 521/700
138/138 - 1s - loss: 0.0100 - MSE: 0.0100 - RMSE: 0.0999 - MAE: 0.0521 - val_loss: 0.0102 - val_MSE: 0.0102 - val_RMSE: 0.1011 - val_MAE: 0.0516
Epoch 522/700
138/138 - 1s - loss: 0.0100 - MSE: 0.0100 - RMSE: 0.0999 - MAE: 0.0520 - val_loss: 0.0104 - val_MSE: 0.0104 - val_RMSE: 0.1018 - val_MAE: 0.0524
Epoch 523/700
138/138 - 1s - loss: 0.0099 - MSE: 0.0099 - RMSE: 0.0996 - MAE: 0.0519 - val_loss: 0.0103 - val_MSE: 0.0103 - val_RMSE: 0.1016 - val_MAE: 0.0520
Epoch 524/700
138/138 - 1s - loss: 0.0099 - MSE: 0.0099 - RMSE: 0.0997 - MAE: 0.0520 - val_loss: 0.0102 - val_MSE: 0.0102 - val_RMSE: 0.1011 - val_MAE: 0.0518
Epoch 525/700
138/138 - 1s - loss: 0.0099 - MSE: 0.0099 - RMSE: 0.0997 - MAE: 0.0518 - val_loss: 0.0102 - val_MSE: 0.0102 - val_RMSE: 0.1010 - val_MAE: 0.0515
Epoch 526/700
138/138 - 1s - loss: 0.0099 - MSE: 0.0099 - RMSE: 0.0993 - MAE: 0.0516 - val_loss: 0.0100 - val_MSE: 0.0100 - val_RMSE: 0.1000 - val_MAE: 0.0533
Epoch 527/700
138/138 - 1s - loss: 0.0099 - MS

Epoch 573/700
138/138 - 1s - loss: 0.0095 - MSE: 0.0095 - RMSE: 0.0972 - MAE: 0.0508 - val_loss: 0.0100 - val_MSE: 0.0100 - val_RMSE: 0.0999 - val_MAE: 0.0509
Epoch 574/700
138/138 - 1s - loss: 0.0095 - MSE: 0.0095 - RMSE: 0.0972 - MAE: 0.0507 - val_loss: 0.0095 - val_MSE: 0.0095 - val_RMSE: 0.0973 - val_MAE: 0.0499
Epoch 575/700
138/138 - 1s - loss: 0.0093 - MSE: 0.0093 - RMSE: 0.0965 - MAE: 0.0504 - val_loss: 0.0094 - val_MSE: 0.0094 - val_RMSE: 0.0968 - val_MAE: 0.0500
Epoch 576/700
138/138 - 1s - loss: 0.0092 - MSE: 0.0092 - RMSE: 0.0959 - MAE: 0.0501 - val_loss: 0.0092 - val_MSE: 0.0092 - val_RMSE: 0.0957 - val_MAE: 0.0500
Epoch 577/700
138/138 - 1s - loss: 0.0092 - MSE: 0.0092 - RMSE: 0.0958 - MAE: 0.0500 - val_loss: 0.0095 - val_MSE: 0.0095 - val_RMSE: 0.0973 - val_MAE: 0.0500
Epoch 578/700
138/138 - 1s - loss: 0.0092 - MSE: 0.0092 - RMSE: 0.0960 - MAE: 0.0501 - val_loss: 0.0095 - val_MSE: 0.0095 - val_RMSE: 0.0974 - val_MAE: 0.0504
Epoch 579/700
138/138 - 1s - loss: 0.0093 - MS

Epoch 625/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0943 - MAE: 0.0491 - val_loss: 0.0089 - val_MSE: 0.0089 - val_RMSE: 0.0942 - val_MAE: 0.0490
Epoch 626/700
138/138 - 1s - loss: 0.0088 - MSE: 0.0088 - RMSE: 0.0939 - MAE: 0.0489 - val_loss: 0.0089 - val_MSE: 0.0089 - val_RMSE: 0.0945 - val_MAE: 0.0486
Epoch 627/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0945 - MAE: 0.0492 - val_loss: 0.0091 - val_MSE: 0.0091 - val_RMSE: 0.0955 - val_MAE: 0.0494
Epoch 628/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0944 - MAE: 0.0491 - val_loss: 0.0090 - val_MSE: 0.0090 - val_RMSE: 0.0948 - val_MAE: 0.0485
Epoch 629/700
138/138 - 1s - loss: 0.0088 - MSE: 0.0088 - RMSE: 0.0940 - MAE: 0.0488 - val_loss: 0.0092 - val_MSE: 0.0092 - val_RMSE: 0.0959 - val_MAE: 0.0489
Epoch 630/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0942 - MAE: 0.0489 - val_loss: 0.0095 - val_MSE: 0.0095 - val_RMSE: 0.0972 - val_MAE: 0.0491
Epoch 631/700
138/138 - 1s - loss: 0.0090 - MS

Epoch 677/700
138/138 - 1s - loss: 0.0085 - MSE: 0.0085 - RMSE: 0.0922 - MAE: 0.0480 - val_loss: 0.0089 - val_MSE: 0.0089 - val_RMSE: 0.0944 - val_MAE: 0.0479
Epoch 678/700
138/138 - 1s - loss: 0.0087 - MSE: 0.0087 - RMSE: 0.0931 - MAE: 0.0481 - val_loss: 0.0090 - val_MSE: 0.0090 - val_RMSE: 0.0950 - val_MAE: 0.0495
Epoch 679/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0944 - MAE: 0.0485 - val_loss: 0.0094 - val_MSE: 0.0094 - val_RMSE: 0.0970 - val_MAE: 0.0478
Epoch 680/700
138/138 - 1s - loss: 0.0089 - MSE: 0.0089 - RMSE: 0.0941 - MAE: 0.0483 - val_loss: 0.0090 - val_MSE: 0.0090 - val_RMSE: 0.0951 - val_MAE: 0.0489
Epoch 681/700
138/138 - 1s - loss: 0.0088 - MSE: 0.0088 - RMSE: 0.0936 - MAE: 0.0486 - val_loss: 0.0088 - val_MSE: 0.0088 - val_RMSE: 0.0937 - val_MAE: 0.0480
Epoch 682/700
138/138 - 1s - loss: 0.0086 - MSE: 0.0086 - RMSE: 0.0925 - MAE: 0.0479 - val_loss: 0.0085 - val_MSE: 0.0085 - val_RMSE: 0.0921 - val_MAE: 0.0474
Epoch 683/700
138/138 - 1s - loss: 0.0085 - MS

In [ ]:
###############################################################################
## Analyze results
###############################################################################
X_val_pred   = clf.predict (X_val)
X_train_pred = clf.predict (X_train)
print ('Train error (MSE):'     , mean_squared_error (X_train_pred, X_train))
print ('Validation error (MSE):', mean_squared_error (X_val_pred, X_val))

#SAMPLES = 50
#print ('Error on first', SAMPLES, 'samples:')
#print ('MSE (pred, real)')
#for pred_sample, real_sample in zip (X_val_pred [:SAMPLES], X_val [:SAMPLES]):
#  print (mean_squared_error (pred_sample, real_sample))


train_mse_element_wise = np.mean (np.square (X_train_pred - X_train), axis = 1)
val_mse_element_wise = np.mean (np.square (X_val_pred - X_val), axis = 1)

In [62]:
max_threshold_val = np.max (val_mse_element_wise)
val_mse_element_wise.sort ()
print ('Maximum validation error (MSE):', max_threshold_val)
print ('Bottom 20 validation error (MSE):', val_mse_element_wise [:20])
print ('Top 20 validation error (MSE):', val_mse_element_wise [::-1][:20])


### K: This looks like another hyperparameter to be adjusted by using a
### separate validation set that contains normal and anomaly samples.
### K: I've guessed 1%, this may be a future line of research.
THRESHOLD_SAMPLE_PERCENTAGE = 1/100
top_n_values_val = np.partition (-val_mse_element_wise,
                                 int (round (val_mse_element_wise.shape [0] *
                                             THRESHOLD_SAMPLE_PERCENTAGE)))

top_n_values_val = -top_n_values_val [: int (round (val_mse_element_wise.shape [0] *
                                                    THRESHOLD_SAMPLE_PERCENTAGE))]
print ('\nNumber of samples considered:', int (round (val_mse_element_wise.shape [0] *
                                        THRESHOLD_SAMPLE_PERCENTAGE)))
top_n_values_val.sort ()
print ('Bottom 20 on considered samples (MSE):', top_n_values_val [:20])
print ('Top 20 on considered samples (MSE):', top_n_values_val [::-1][:20])


### K: O limiar de classificacao sera a mediana dos N maiores custos obtidos
### ao validar a rede no conjunto de validacao. N e um hiperparametro que pode
### ser ajustado, mas e necessario um conjunto de validacao com amostras
### anomalas em adicao ao conjunto de validacao atual, que so tem amostras nao
### anomalas. @TODO: Desenvolver e validar o conjunto com esta nova tecnica.
threshold = np.median (top_n_values_val)
print ('Thresh val:', threshold)

Maximum validation error (MSE): 113.2305626946235
Bottom 20 validation error (MSE): [9.65774621e-05 9.65813386e-05 9.66101934e-05 9.66196180e-05
 9.66270676e-05 9.66280150e-05 9.67970847e-05 9.68319353e-05
 9.68381338e-05 9.68435115e-05 9.68609315e-05 9.81716281e-05
 9.83804827e-05 9.88136386e-05 9.92726595e-05 9.96991779e-05
 9.97331073e-05 9.97350764e-05 9.97420213e-05 9.99043797e-05]
Top 20 validation error (MSE): [113.23056269  67.74157643  40.97403839  35.68251822  32.55987208
  31.69926558  23.96302262  22.09646066  20.08897216  19.13668474
  17.62034362  16.11111278  15.8275275   12.51100187  12.30916942
  11.7822027   10.625362    10.55641148  10.51409532  10.09469521]

Number of samples considered: 4584
Bottom 20 on considered samples (MSE): [0.07692614 0.07692742 0.0769276  0.07692893 0.07692899 0.07692974
 0.07697721 0.07702407 0.07703011 0.07703467 0.07704096 0.07705148
 0.07706041 0.07711004 0.07711528 0.07712873 0.07713381 0.07713715
 0.07713941 0.07714035]
Top 20 on cons

In [ ]:
### K: NOTE: Only look at test results when publishing...
sys.exit ()
X_test_pred = clf.predict (X_test)
print (X_test_pred.shape)
print ('Test error:', mean_squared_error (X_test_pred, X_test))


y_pred = np.mean (np.square (X_test_pred - X_test), axis = 1)
#y_pred = []
#for pred_sample, real_sample, label in zip (X_test_pred, X_test, y_test):
#  y_pred.append (mean_squared_error (pred_sample, real_sample))

#print ('\nLabel | MSE (pred, real)')
#for label, pred in zip (y_test, y_pred):
#  print (label, '|', pred)

y_test, y_pred = zip (*sorted (zip (y_test, y_pred)))
#print ('\nLabel | MSE (pred, real) (ordered)')
#for label, pred in zip (y_test, y_pred):
#  print (label, '|', pred)

# 0 == normal
# 1 == attack
print ('\nPerformance on TEST set:')
print ('\nMSE (pred, real) | Label (ordered)')
tp, tn, fp, fn = 0, 0, 0, 0
for label, pred in zip (y_test, y_pred):
#  if (pred >= threshold):
#    print ('Classified as anomaly     (NORMAL):', label)
#  else:
#    print ('Classified as not anomaly (ATTACK):', label)

  if ((pred >= threshold) and (label == 0)):
    print ('True negative.')
    tn += 1
  elif ((pred >= threshold) and (label == 1)):
    print ('False negative!')
    fn += 1
  elif ((pred < threshold) and (label == 1)):
    print ('True positive.')
    tp += 1
  elif ((pred < threshold) and (label == 0)):
    print ('False positive!')
    fp += 1

print ('Confusion matrix:')
print ('tp | fp')
print ('fn | tn')
print (tp, '|', fp)
print (fn, '|', tn)
print ('TP:', tp)
print ('TN:', tn)
print ('FP:', fp)
print ('FN:', fn)